In [5]:
import TradeClient
import asyncio
import nest_asyncio
import Analysis_new
import MarketDataFetcher
import DataStoreage
from typing import List, Union, Final
nest_asyncio.apply()

ins_market = MarketDataFetcher.FuturesMarket()

symbols = ['BTCUSDT', 'TRXUSDT', 'XRPUSDT', 'ADAUSDT', 'ETHUSDT']
data = {}
for symbol in symbols:
    data[symbol] = DataStoreage.KlineData()
    kline_data = asyncio.run(ins_market.fetch_klines_limit(symbol=symbol, interval='3m', limit=100))
    data[symbol].set_data(kline_data)

ins_analysis = Analysis_new.AnalysisManager(data)
ins_analysis.get_success_signal()

{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0]}}
{1: {}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0], 'TRXUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1: {'BTCUSDT': [0, 1, 1, 0], 'TRXUSDT': [0, 1, 1, 0], 'XRPUSDT': [0, 1, 1, 0]}, 2: {'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}}
{1

In [6]:
print(ins_analysis.ins_sell_strategy_1.result)
print(ins_analysis.ins_buy_strategy_1.result)

{'BTCUSDT': [0, 2, 1, 0], 'TRXUSDT': [0, 2, 1, 0], 'XRPUSDT': [0, 2, 1, 0], 'ADAUSDT': [0, 2, 1, 0], 'ETHUSDT': [0, 2, 1, 0]}
{'BTCUSDT': [0, 1, 1, 0], 'TRXUSDT': [0, 1, 1, 0], 'XRPUSDT': [0, 1, 1, 0], 'ADAUSDT': [0, 1, 1, 0], 'ETHUSDT': [0, 1, 1, 0]}


In [9]:
from dataclasses import dataclass
from typing import Optional, Dict, List, Any, Union, Final
import utils

@dataclass
class TradingLog:
    """
    Position 진입 주문 시 관련 정보를 기록하며, 이 데이터를 기반으로 거래 내역 및 현재 지갑 상태를 관리한다.
    힌트에서 Optional(None)으로 설정된 항목들은 거래 종료 후 또는 사후에 추가될 데이터들을 나타내며,
    속성값 배열의 정렬을 고려하여 기본값을 None으로 미설정 및 세부 설정은 삭제하였다.
    """
    
    ### 주문 관련 정보
    symbol: str  # 심볼 (예: BTCUSDT)
    position_type: int  # 포지션 유형 (1: Long, 2: Short)
    quantity: float  # 주문 수량
    hedge_enable: bool  # 헤지 여부 (진행 중인 포지션과 반대 방향의 주문 여부)
    leverage: int  # 레버리지 배율
    
    ### 가격 정보
    open_price: float  # 진입 가격 (Open Price)
    high_price: float  # 최고 가격
    low_price: float  # 최저 가격
    close_price: float  # 현재 가격 (Close Price)
    
    ### 시간 정보
    start_timestamp: int  # 시작 타임스탬프
    end_timestamp: Optional[int]=None  # 현재 시간 타임스탬프(포지션 종료 시점)
    
    ### 손절 및 종료 설정
    scale_stop_enable: bool = True  # final 손절율 or scale 손절율 적용 여부
    initial_stop_rate:float = 0.015  # 초기 손절 비율
    trailing_stop_rate: float = 0.65  # 트레일링 손절 비율 (후기 손절 비율)
    reverse_position_ratio: float = 0  # 포지션과 반대의 흐름 비율 (반대 캔들 비율)
    time_based_adjustment_rate: float = 0.0007  # 시간 흐름에 따른 비율 조정 값
    adjusted_interval:str = '3m'    # 조종 변동 step
    adjusted_entry_price: Optional[float]=None  # 조정된 진입 가격 (StopLoss 기준)
    stop_trigger_enable: bool=False      # 종료 신호 활성화 여부
    stop_loss_price: Optional[float]=None  # 손절 가격 또는 종료 가격

    ### 포지션 평가
    initial_value: Optional[float]=None  # 진입 시점의 평가 가치 (수수료 제외)
    current_value: Optional[float]=None  # 현재(종료) 시점의 평가 가치 (수수료 제외)
    net_pnl: Optional[float]=None  # 순 손익 금액 (Net Profit or Loss, 수수료 제외)
    net_pnl_rate: Optional[float]=None  # 순 손익 비율 (Net Profit or Loss Rate, 수수료 제외)
    gross_pnl: Optional[float]=None  # 총 손익 금액 (Gross Profit or Loss, 수수료 포함)
    gross_pnl_rate: Optional[float]=None  # 총 손익 비율 (Gross Profit or Loss Rate, 수수료 포함)
    
    ### 시스템 세팅
    test_mode: bool = False
    
    ### 수수료 관련
    entry_fee: float=0  # 진입 수수료
    exit_fee: float=0  # 종료 수수료
    
    def __post_init__(self):
        """
        TradingLog 선언시 진입금액과 현재금액을 계산한다.
        """
        # 진입금액(마진)을 계산한다.
        if self.initial_value is None:
            self.initial_value = (self.open_price * self.quantity) / self.leverage
        # 현재금액을 계산한다.
        if self.current_value is None:
            self.current_value = (self.close_price * self.quantity) / self.leverage
    
    def update_trade_data(self, timestamp:int, price:Optional[Union[float, int]]=None):
        """
        신규 데이터를 TradingLog데이터에 반영 및 연산한다.
        """
        # 현재시간을 업데이트한다.
        self.end_timestamp = timestamp
        # price 데이터 입력시 포지션에 맞게 high_price or low_price를 업데이트 한다.
        if price is not None:
            if self.position_type == 1:
                self.high_price = max(self.high_price, price)
            elif self.position_type == 2:
                self.low_price = min(self.low_price, price)
        
        # 현재 평가금액을 계산한다.
        self.__cals_value()
        # 손절 또는 종료연산에 필요한 값을 계산하고 포지션 종료여부를 결정한다.
        self.__cals_stop_loss()
        # self.test_mode가 True일 경우에만 동작하며 수수료(진입/종료)를 계산한다.
        self.__cals_fees()
    
    def __cals_fees(self):
        """
        self.test_mode가 True일 경우에만 수수료를 계산한다. 백테스트 전용 함수다.
        """
        # 수수료는 시장가 기준으로 0.05%를 적용했으며, 정확한 공식 적용이 어려우므로 슬리피지는 별도 적용하지 않는다.
        FEE_RATE:Final[float] = 0.0005
        # 테스트 모드 여부를 확인한다.
        if self.test_mode:
            # 진입 수수료를 계산한다.
            self.entry_fee = (self.open_price * self.quantity * FEE_RATE)
            # 종료 수수료를 계산한다.
            self.exit_fee = (self.close_price * self.quantity * FEE_RATE)
    
    def __cals_value(self):
        """
        현재 평가금액, PNL을 계산한다. 
        """
        
        # 현재 평가금액 계산한다.
        self.current_value = (self.close_price * self.quantity) / self.leverage
        
        # 포지션에 따라서 수수료 제외한 pnl을 계산한다.
        if self.position_type == 1:
            self.net_pnl = self.current_value - self.initial_value
        elif self.position_type == 2:
            self.net_pnl = self.initial_value - self.current_value
        # 수수료 제외한 pnl의 비율을 계산한다.
        self.net_pnl_rate = self.net_pnl / self.initial_value
        # 수수료 총 합계를 계산한다.
        total_fee = self.entry_fee + self.exit_fee
        # 수수료 비용을 포함한 pnl을 계산한다.
        self.gross_pnl = self.net_pnl - total_fee
        # 수수료 비용을 포함한 pnl비율을 계산한다.
        self.gross_pnl_rate = self.gross_pnl / self.initial_value
    
    
    def __cals_stop_loss(self):
        """
        스탑 또는 종료 관련값을 계산한다.
        """
        # interval값에 대한 밀리초 값을 구한다.
        target_ms_seconds = utils._get_interval_ms_seconds(self.adjusted_interval)
        # interval값 오입력시 프로그램 종료한다.
        if target_ms_seconds is None:
            # interval 값은 바이낸스 interval값을 기준으로 한다.
            raise ValueError(f'interval값이 유효하지 않음:{self.adjusted_interval}')
        # 종료 타임스템프와 시작타임스템프의 차이를 구한다.
        time_diff = self.end_timestamp - self.start_timestamp
        # 거래발생부터 현재시간을 interval값으로 나누어 횟수를 구하고 지정된 비율을 곱하여 추가적용할 비율을 계산한다.
        dynamic_time_rate = int(time_diff / target_ms_seconds) * self.time_based_adjustment_rate
        
        # 포지션이 롱일경우 스탑로스를 계산한다.
        if self.position_type == 1:
            # 설정상 scale_stop_enable이 참 일경우
            if self.scale_stop_enable:
                # 스탑로스 비율을 계산한다.
                stop_loss_rate = self.initial_stop_rate - (self.reverse_position_ratio + dynamic_time_rate)
                # 유동적 시작가를 계산한다.
                self.adjusted_entry_price = self.open_price * (1-stop_loss_rate)
                # 포지션을 종료 금액을 계산한다.
                self.stop_loss_price = self.adjusted_entry_price + ((self.high_price - self.adjusted_entry_price) * (1-stop_loss_rate))
            # 설정상 scale_stop_enable이 거짓일 경우
            else:
                # 포지션 종료 금액을 계산한다.
                self.stop_loss_price = self.high_price * (1-self.trailing_stop_rate)
            # 현재가격이 포지션 종료금액 이하시 True변환
            self.stop_trigger_enable = True if self.stop_loss_price <= self.close_price else False

        # 포지션이 숏일경우 스탑로스를 계산한다.
        elif self.position_type == 2:
            # 설정상 scale_stop_enable이 참 일경우
            if self.scale_stop_enable:
                # 스탑로스 비율을 계산한다.
                stop_loss_rate = self.initial_stop_rate - (self.reverse_position_ratio + dynamic_time_rate)
                # 유동적 시작가를 계산한다.
                self.adjusted_entry_price = self.open_price * (1+stop_loss_rate)
                # 포지션을 종료 금액을 계산한다.
                self.stop_loss_price = self.adjusted_entry_price - ((self.adjusted_entry_price - self.low_price) * (1-stop_loss_rate))
            
            else:
                # 포지션 종료 금액을 계산한다.
                self.stop_loss_price = self.low_price * (1+self.trailing_stop_rate)
            # 현재가격이 포지션 종료금액 이상시 True 반환
            self.stop_trigger_enable = True if self.stop_loss_price >= self.close_price else False

                
                
from pprint import pprint
import time
    
trade_log = TradingLog(
    symbol='BTCUSDT',
    start_timestamp=int(time.time()*1000)-(60000*3*60),
    end_timestamp=None,
    position_type=1,
    quantity=123.1,
    hedge_enable=False,
    leverage=15,
    open_price=100,
    high_price=100,
    low_price=100,
    close_price=100,
    test_mode=True
)

pprint(trade_log)

TradingLog(symbol='BTCUSDT',
           position_type=1,
           quantity=123.1,
           hedge_enable=False,
           leverage=15,
           open_price=100,
           high_price=100,
           low_price=100,
           close_price=100,
           start_timestamp=1738094838212,
           end_timestamp=None,
           scale_stop_enable=True,
           initial_stop_rate=0.015,
           trailing_stop_rate=0.65,
           reverse_position_ratio=0,
           time_based_adjustment_rate=0.0007,
           adjusted_interval='3m',
           adjusted_entry_price=None,
           stop_trigger_enable=False,
           stop_loss_price=None,
           initial_value=820.6666666666666,
           current_value=820.6666666666666,
           net_pnl=None,
           net_pnl_rate=None,
           gross_pnl=None,
           gross_pnl_rate=None,
           test_mode=True,
           entry_fee=0,
           exit_fee=0)


In [10]:
trade_log.update_trade_data(timestamp=int(time.time()*1000)-(60000*3*20), price=200)
pprint(trade_log)

TradingLog(symbol='BTCUSDT',
           position_type=1,
           quantity=123.1,
           hedge_enable=False,
           leverage=15,
           open_price=100,
           high_price=200,
           low_price=100,
           close_price=100,
           start_timestamp=1738094838212,
           end_timestamp=1738102038415,
           scale_stop_enable=True,
           initial_stop_rate=0.015,
           trailing_stop_rate=0.65,
           reverse_position_ratio=0,
           time_based_adjustment_rate=0.0007,
           adjusted_interval='3m',
           adjusted_entry_price=101.29999999999998,
           stop_trigger_enable=False,
           stop_loss_price=201.2831,
           initial_value=820.6666666666666,
           current_value=820.6666666666666,
           net_pnl=0.0,
           net_pnl_rate=0.0,
           gross_pnl=0.0,
           gross_pnl_rate=0.0,
           test_mode=True,
           entry_fee=6.155,
           exit_fee=6.155)


In [11]:
trade_log.update_trade_data(timestamp=int(time.time()*1000), price=200)
pprint(trade_log)

TradingLog(symbol='BTCUSDT',
           position_type=1,
           quantity=123.1,
           hedge_enable=False,
           leverage=15,
           open_price=100,
           high_price=200,
           low_price=100,
           close_price=100,
           start_timestamp=1738094838212,
           end_timestamp=1738105638642,
           scale_stop_enable=True,
           initial_stop_rate=0.015,
           trailing_stop_rate=0.65,
           reverse_position_ratio=0,
           time_based_adjustment_rate=0.0007,
           adjusted_interval='3m',
           adjusted_entry_price=102.69999999999999,
           stop_trigger_enable=False,
           stop_loss_price=202.62709999999998,
           initial_value=820.6666666666666,
           current_value=820.6666666666666,
           net_pnl=0.0,
           net_pnl_rate=0.0,
           gross_pnl=-12.31,
           gross_pnl_rate=-0.015000000000000001,
           test_mode=True,
           entry_fee=6.155,
           exit_fee=6.155)


In [12]:
trade_log.reverse_position_ratio = 0.2
trade_log.update_trade_data(timestamp=int(time.time()*1000), price=200)
pprint(trade_log)

TradingLog(symbol='BTCUSDT',
           position_type=1,
           quantity=123.1,
           hedge_enable=False,
           leverage=15,
           open_price=100,
           high_price=200,
           low_price=100,
           close_price=100,
           start_timestamp=1738094838212,
           end_timestamp=1738105638863,
           scale_stop_enable=True,
           initial_stop_rate=0.015,
           trailing_stop_rate=0.65,
           reverse_position_ratio=0.2,
           time_based_adjustment_rate=0.0007,
           adjusted_interval='3m',
           adjusted_entry_price=122.7,
           stop_trigger_enable=False,
           stop_loss_price=217.5471,
           initial_value=820.6666666666666,
           current_value=820.6666666666666,
           net_pnl=0.0,
           net_pnl_rate=0.0,
           gross_pnl=-12.31,
           gross_pnl_rate=-0.015000000000000001,
           test_mode=True,
           entry_fee=6.155,
           exit_fee=6.155)


In [13]:
0.0007*20

0.014

In [ ]:
import numpy as np
a = [1,2,3,4,5]
array = np.array(a)
np.where(array>1)

In [ ]:
np.where(array>1)

In [ ]:
vars(ins_analysis).keys()

In [2]:
signal ={1:[], 2:[]}


In [3]:
signal

{1: [], 2: []}